In [ ]:
import folium
import pandas as pd

In [ ]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [ ]:
import pandas as pd
import requests
import io

# Define the URL
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

# Fetch the data
response = requests.get(URL)
response.raise_for_status()  # Ensure we notice bad responses

# Load the CSV into a Pandas DataFrame
spacex_df = pd.read_csv(io.StringIO(response.text))

# Display the first few rows
spacex_df.head()


   Flight Number        Date Time (UTC) Booster Version  Launch Site  \
0              1  2010-06-04   18:45:00  F9 v1.0  B0003  CCAFS LC-40   
1              2  2010-12-08   15:43:00  F9 v1.0  B0004  CCAFS LC-40   
2              3  2012-05-22    7:44:00  F9 v1.0  B0005  CCAFS LC-40   
3              4  2012-10-08    0:35:00  F9 v1.0  B0006  CCAFS LC-40   
4              5  2013-03-01   15:10:00  F9 v1.0  B0007  CCAFS LC-40   

                                             Payload  Payload Mass (kg)  \
0               Dragon Spacecraft Qualification Unit                0.0   
1  Dragon demo flight C1, two CubeSats,  barrel o...                0.0   
2                             Dragon demo flight C2+              525.0   
3                                       SpaceX CRS-1              500.0   
4                                       SpaceX CRS-2              677.0   

       Orbit         Customer        Landing Outcome  class        Lat  \
0        LEO           SpaceX  Failure   (

In [ ]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [ ]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [ ]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [ ]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

for _,row in launch_sites_df.iterrows():
  color = "red"
  folium.Marker(location=[row["Lat"], row["Long"]],popup=f"{row['Launch Site']}", icon=folium.Icon(color=color, icon="rocket", prefix="fa" )).add_to(site_map)

site_map

In [ ]:
launch_summary = spacex_df.groupby("Launch Site").agg(
    successful_count=("class", lambda x: (x == 1).sum()),
    failed_count=("class", lambda x: (x == 0).sum()),
    Lat=("Lat", "first"),
    Long=("Long", "first")
).reset_index()

In [ ]:
launch_summary

,Launch Site,successful_count,failed_count,Lat,Long
0,CCAFS LC-40,7,19,28.562302,-80.577356
1,CCAFS SLC-40,3,4,28.563197,-80.576820
2,KSC LC-39A,10,3,28.573255,-80.646895
3,VAFB SLC-4E,4,6,34.632834,-120.610745


In [ ]:
for _, row in launch_summary.iterrows():
    popup_text = f"{row['Launch Site']}\nSuccess: {row['successful_count']}\nFailure: {row['failed_count']}"
    folium.Marker(
        location=[row["Lat"], row["Long"]],
        popup=popup_text,
        icon=folium.Icon(color="blue", icon="rocket", prefix="fa")
    ).add_to(site_map)
site_map


In [ ]:
from folium.plugins import MarkerCluster
# Add marker color column based on launch success/failure
spacex_df["marker_color"] = spacex_df["class"].apply(lambda x: "green" if x == 1 else "red")

# Create MarkerCluster object
marker_cluster = MarkerCluster()

# Add markers to cluster
for _, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record["Lat"], record["Long"]],
        popup=f"{record['Launch Site']} - {'Success' if record['class'] == 1 else 'Failure'}",
        icon=folium.Icon(color=record["marker_color"], icon="rocket", prefix="fa")
    )
    marker_cluster.add_child(marker)

# Add MarkerCluster to map
site_map.add_child(marker_cluster)

# Save the map as an HTML file
site_map.save("spacex_launch_sites.html")

# Display map (for Jupyter Notebook users)
site_map


In [ ]:
import folium
from folium.plugins import MousePosition
from math import sin, cos, sqrt, atan2, radians

# Haversine Distance Function
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# SpaceX Launch Sites
launch_sites = [
    {"name": "KSC LC-39A", "lat": 28.5721, "lon": -80.6480},
    {"name": "CCAFS SLC-40", "lat": 28.5623, "lon": -80.5774},
    {"name": "VAFB SLC-4E", "lat": 34.6328, "lon": -120.6108},
    {"name": "Boca Chica", "lat": 25.9972, "lon": -97.1566},
]

# Create Folium Map
site_map = folium.Map(location=[28.5721, -80.6480], zoom_start=5)

# Add Launch Site Markers
for site in launch_sites:
    folium.Marker(
        [site["lat"], site["lon"]],
        popup=site["name"],
        icon=folium.Icon(color="blue", icon="rocket"),
    ).add_to(site_map)

# Mouse Position Plugin to Show Coordinates
MousePosition().add_to(site_map)

# LatLngPopup - Click to Get Coordinates
folium.LatLngPopup().add_to(site_map)

# Function to Find Nearest Site and Display Distance
def add_distance_markers(map_object, lat, lon):
    min_dist = float("inf")
    nearest_site = None

    for site in launch_sites:
        dist = calculate_distance(lat, lon, site["lat"], site["lon"])
        if dist < min_dist:
            min_dist = dist
            nearest_site = site

    # Add Marker at Clicked Location
    folium.Marker(
        [lat, lon],
        popup=f"Clicked Location: {lat:.4f}, {lon:.4f}<br>"
              f"Nearest Site: {nearest_site['name']}<br>"
              f"Distance: {min_dist:.2f} km",
        icon=folium.Icon(color="red"),
    ).add_to(map_object)

# Save Map as HTML
site_map.save("spacex_click_distance_map.html")
site_map

